In [1]:
import matplotlib.pyplot as plt
import open3d as o3d
import numpy as np
import os

import pyFM.spectral as LB
from pyFM.mesh import TriMesh

import polyscope as ps
ps.init()

import meshplot as mp
def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot_o3d(myMesh1,myMesh2,cmap1=None,cmap2=None):
    vertlist_1 = np.asarray(myMesh1.vertices)
    facelist_1 = np.asarray(myMesh1.triangles)

    vertlist_2 = np.asarray(myMesh2.vertices)
    facelist_2 = np.asarray(myMesh2.triangles)
    
    d = mp.subplot(vertlist_1, facelist_1, c=cmap1, s=[2, 2, 0])
    mp.subplot(vertlist_2, facelist_2, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3


In [5]:
mesh_1 = o3d.io.read_triangle_mesh(os.path.abspath("../simulation/meshes_dancer_flattened/frame_0001.ply"))
mesh_2 = o3d.io.read_triangle_mesh(os.path.abspath("../simulation/meshes_dancer_flattened/frame_0002.ply"))

mesh_1.compute_vertex_normals()
mesh_2.compute_vertex_normals()

verts_1 = np.asarray(mesh_1.vertices)
verts_2 = np.asarray(mesh_2.vertices)

faces_1 = np.asarray(mesh_1.triangles)
vertices_1 = np.asarray(mesh_1.vertices)
ps_mesh_1 = ps.register_surface_mesh("mesh_1", vertices_1, faces_1)

faces_2 = np.asarray(mesh_2.triangles)
vertices_2 = np.asarray(mesh_2.vertices)
ps_mesh_2 = ps.register_surface_mesh("mesh_2", vertices_2, faces_2)
ps.show()
ps.remove_all_structures() # clear the whole scene

#o3d.visualization.draw_geometries([mesh_1] + [mesh_2])

## KDTREE Nearest Neighbour Seach with Open3D

In [ ]:
# KDTree for mesh_1 -> mesh_2
pcd_2 = o3d.geometry.PointCloud() # create pointcloud
pcd_2.points = o3d.utility.Vector3dVector(verts_2) # assign verts with normals to cloud
kdtree = o3d.geometry.KDTreeFlann(pcd_2)

# search for nearest neighbour in mesh_1
p2p_map = []
for vert in verts_1:
    [_, idx, _] = kdtree.search_knn_vector_3d(vert, 1) 
    p2p_map.append(idx[0])

p2p_map = np.array(p2p_map)

Why is the map showing up wrong

In [7]:
cmap_1 = visu(verts_1)
cmap_2 = cmap_1[p2p_map]

double_plot_o3d(mesh_1, mesh_2, cmap_1,cmap_2)

was just trying different ways to visualise in polyscope below (not working well) 

In [91]:
faces_1 = np.asarray(mesh_1.triangles)
vertices_1 = np.asarray(mesh_1.vertices)
ps_mesh_1 = ps.register_surface_mesh("mesh_1", vertices_1, faces_1)

faces_2 = np.asarray(mesh_2.triangles)
vertices_2 = np.asarray(mesh_2.vertices)
ps_mesh_2 = ps.register_surface_mesh("mesh_2", vertices_2, faces_2)

# cmaps
# cmap_1 = (np.linspace(0,1,len(vertices_1))).reshape(-1,1) # gradient for mesh_1
# cmap_1 = np.repeat(cmap_1, 3, axis=1) 
# print(cmap_1)

# cmap_2 = np.zeros((len(vertices_2), 3))
# for i, j in enumerate(p2p_map):
#     cmap_2[j] = cmap_1[i]   # assign color of vertex i in mesh1 to vertex j in mesh2
# ------------------------------------------------------------------------------------------
cmap = plt.get_cmap("jet") # RGB ALpha colour map from matplotlib
# assign scalar values to mesh1
scalars_1 = np.linspace(0, 1, len(verts_1))

# map mesh1 scalars into mesh2 using correspondence
scalars_2 = np.zeros(len(verts_2))   # default zero for all verts
scalars_2[p2p_map] = scalars_1       # assign scalars where correspondence exists

cmap_1 = cmap(scalars_1)[:,:3]
cmap_2 = cmap(scalars_2)[:,:3]

ps_mesh_1.add_color_quantity("cmap_1", cmap_1, enabled=True)
ps_mesh_2.add_color_quantity("cmap_2", cmap_2, enabled=True)

ps.show()
ps.remove_all_structures() # clear the whole scene

In [8]:
num_samples = 100
sample_idx = np.random.choice(len(vertices_1), num_samples, replace=False)

edges = []
points = []

# stack both meshes in one point cloud so we can draw edges between them
points = np.vstack([vertices_1, vertices_2])

# edges connect vertex i in mesh1 → corresponding vertex in mesh2
for i in sample_idx:
    edges.append([i, len(vertices_1) + p2p_map[i]])

edges = np.array(edges)

ps.register_curve_network("Correspondences", points, edges, radius=0.002, color=(0,0,0))
ps.show()
ps.remove_all_structures() # clear the whole scene

## Converting to use in pyFM

1. convert meshes to TriMesh
2. use k nearest neighbour search to find the maps
3. find eigvects and assign to the mesh
4. convert to spectral map to be used in ConsistentZoomout
5. save to maps_dict

do I need to save the map dicts in a file I can load later?

In [ ]:
K = 30 # dimensions used in the pyFM examples when they convert to functional maps
k_eigs = 150 # the pyFM examples use 150 when they load their mesh
# eigvects_1, eigvals_1 = LB(mesh_1, k_eigs)
# eigvects_2, eigvals_2 = LB(mesh_2, k_eigs)

C_init = LB.mesh_p2p_to_FM(p2p_map, mesh_1, mesh_2, dims=K)

cmap1 = visu(mesh_1.vertlist); cmap2 = cmap1[p2p_map]
double_plot_o3d(mesh_1, mesh_2, cmap1, cmap2)

AttributeError: 'open3d.cpu.pybind.geometry.TriangleMesh' object has no attribute 'eigenvectors'

In [ ]:
# KDTREE SEARCH ON pyFM TriMesh instead of the open3d TriangleMesh 

import open3d as o3d
import numpy as np
from pyFM.mesh import TriMesh

# pyFM mesh
mesh_pyfm = TriMesh(vertices, faces)

# Build Open3D PointCloud for KDTree
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(mesh_pyfm.vertlist)

kdtree = o3d.geometry.KDTreeFlann(pcd)

# Example: nearest neighbour search
query_point = mesh_pyfm.vertlist[0]
_, idx, _ = kdtree.search_knn_vector_3d(query_point, 1)

print("Closest index:", idx[0])

# wait it this one even right??????????????????????/